In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd

import copy
import traceback
import datetime
import joblib
import re
import os
import random
import string
import time

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchtext import data
from torchtext import datasets
from torchtext import vocab
from torchtext.vocab import Vectors, GloVe

from tqdm.notebook import tqdm

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import pymorphy2
from sklearn.base import BaseEstimator, TransformerMixin

In [33]:
class ConfigExperiment:
    seed = 42
    positive_file = "../data/positive.csv"
    negative_file = "../data/negative.csv"
    test_size = 0.3
    device = "cuda" if torch.cuda.is_available() else "cpu"
    embed_dim = 300
    max_vocab_size = 50_000
    batch_size = 64
    num_epochs = 30
    lr = 1e-2
    num_workers = 0
    patience = 3
    early_stopping_delta = 1e-4
    save_dirname = "models"
    
config = ConfigExperiment()

In [34]:
def init_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic=True
    
init_random_seed(config.seed)

In [35]:
# Формирование train test valid данных

df = pd.read_csv("../data/preprocessed_text_v1.csv", index_col=False)
df.columns = ['text', 'target']
df = df.drop(df[df['text'].map(str) == 'nan'].index)

train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
train.to_csv("../data/train_processed_data.csv", index=False)
validate.to_csv("../data/validate_processed_data.csv", index=False)
test.to_csv("../data/test_processed_data.csv", index=False)

train.shape, validate.shape, test.shape

((135615, 2), (45205, 2), (45205, 2))

In [36]:
tokenize = lambda x: str(x).split()

TEXT = data.Field(sequential=True, tokenize=tokenize, batch_first = True)
LABEL = data.LabelField(dtype=torch.float)

fields = [('text',TEXT), ('label', LABEL)]

In [37]:

train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path="../data/",
                                        train="train_processed_data.csv",
                                        validation="validate_processed_data.csv",
                                        test="test_processed_data.csv",
                                        format="csv",
                                        fields=fields,
                                        skip_header=True
)

In [38]:
print(vars(train_data[0]))

{'text': ['блин', 'улица', 'плюсовой', 'температура', 'зря', 'шуба', 'доставать'], 'label': '0'}


In [39]:
print(vars(valid_data[0]))

{'text': ['час', 'спасть'], 'label': '0'}


In [40]:
print(vars(test_data[0]))

{'text': ['шикарный', 'сибирский', 'кот', 'симб', 'справиться', 'быть', 'выпускать', 'жаль'], 'label': '0'}


In [41]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 135615
Number of validation examples: 45205
Number of testing examples: 45205


In [42]:
TEXT.build_vocab(train_data, max_size=config.max_vocab_size)
LABEL.build_vocab(train_data)

In [43]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 50002
Unique tokens in LABEL vocabulary: 2


In [44]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    sort_key = lambda x: x.text,
    batch_size=config.batch_size,
    device=config.device)

In [45]:
# print('Train:')
# for batch in train_iterator:
#     print(batch)
    
# print('Valid:')
# for batch in valid_iterator:
#     print(batch)
    
# print('Test:')
# for batch in test_iterator:
#     print(batch)

In [46]:
print(TEXT.vocab.freqs.most_common(20))

[('хотеть', 7002), ('весь', 6334), ('день', 6002), ('мочь', 5554), ('такой', 5356), ('сегодня', 5297), ('очень', 4601), ('быть', 4362), ('ты', 4251), ('один', 4175), ('просто', 4154), ('год', 4065), ('мой', 4025), ('хороший', 3979), ('человек', 3688), ('знать', 3554), ('любить', 3349), ('завтра', 3189), ('свой', 2902), ('вообще', 2901)]


In [47]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'хотеть', 'весь', 'день', 'мочь', 'такой', 'сегодня', 'очень', 'быть']


In [48]:
print(LABEL.vocab.stoi)

defaultdict(None, {'1': 0, '0': 1})


In [49]:
import zipfile
import gensim
import wget

model_url = 'http://vectors.nlpl.eu/repository/11/187.zip'
# wget.download(model_url)
w2v_model = gensim.models.KeyedVectors.load('187/model.model')
numpy_embeddings = np.zeros(shape=[len(TEXT.vocab), config.embed_dim],dtype=np.float32)

for word in TEXT.vocab.itos:
    vector = w2v_model.get_vector(word)
    index  = TEXT.vocab.stoi[word]
    numpy_embeddings[index] = vector
    
pretrained_embeddings = torch.Tensor(numpy_embeddings).float()
pretrained_embeddings.shape

torch.Size([50002, 300])

In [24]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        #text = [batch size, sent len]
        embedded = self.embedding(text)
        #embedded = [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1)
        #embedded = [batch size, 1, sent len, emb dim]
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        #pooled_n = [batch size, n_filters]
        cat = self.dropout(torch.cat(pooled, dim = 1))
        #cat = [batch size, n_filters * len(filter_sizes)]
        return self.fc(cat)

In [50]:
class CNN1d(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim, 
                                              out_channels = n_filters, 
                                              kernel_size = fs)
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.permute(0, 2, 1)
        
        #embedded = [batch size, emb dim, sent len]
        
        conved = [F.relu(conv(embedded)) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))
        
        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [51]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = config.embed_dim
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN1d(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [52]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 15,361,201 trainable parameters


In [53]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0306, -0.3523, -0.1600,  ..., -0.6755,  0.1621, -0.6484],
        [ 0.5594,  0.7161, -0.0522,  ..., -0.3110,  1.0994, -0.7446],
        [ 1.3671,  0.1885, -1.3516,  ...,  0.9948, -1.3568, -1.2967],
        ...,
        [ 1.2550,  1.1039, -0.8221,  ...,  0.4918, -1.7711, -1.0247],
        [-0.4411, -0.4301, -0.3610,  ..., -1.4883,  0.1232, -0.0570],
        [ 0.0637,  0.1167, -1.3399,  ..., -1.9377,  0.5263, -1.2968]])

In [54]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [56]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(config.device)
criterion = criterion.to(config.device)

In [57]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [58]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [59]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [60]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [61]:
N_EPOCHS = 20

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 26s
	Train Loss: 0.625 | Train Acc: 66.22%
	 Val. Loss: 0.569 |  Val. Acc: 70.37%
Epoch: 02 | Epoch Time: 0m 26s
	Train Loss: 0.567 | Train Acc: 70.99%
	 Val. Loss: 0.565 |  Val. Acc: 70.76%
Epoch: 03 | Epoch Time: 0m 26s
	Train Loss: 0.531 | Train Acc: 73.95%
	 Val. Loss: 0.606 |  Val. Acc: 69.98%
Epoch: 04 | Epoch Time: 0m 26s
	Train Loss: 0.497 | Train Acc: 76.35%
	 Val. Loss: 0.597 |  Val. Acc: 70.72%
Epoch: 05 | Epoch Time: 0m 26s
	Train Loss: 0.465 | Train Acc: 78.13%
	 Val. Loss: 0.603 |  Val. Acc: 70.66%
Epoch: 06 | Epoch Time: 0m 27s
	Train Loss: 0.437 | Train Acc: 79.78%
	 Val. Loss: 0.623 |  Val. Acc: 70.69%
Epoch: 07 | Epoch Time: 0m 26s
	Train Loss: 0.413 | Train Acc: 80.86%
	 Val. Loss: 0.679 |  Val. Acc: 70.15%
Epoch: 08 | Epoch Time: 0m 27s
	Train Loss: 0.389 | Train Acc: 82.17%
	 Val. Loss: 0.695 |  Val. Acc: 69.81%
Epoch: 09 | Epoch Time: 0m 27s
	Train Loss: 0.367 | Train Acc: 83.28%
	 Val. Loss: 0.729 |  Val. Acc: 70.14%
Epoch: 10 | Epoch T

In [36]:
model.load_state_dict(torch.load('tut4-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.333 | Test Acc: 85.73%


In [23]:
# for CNN2d 
# Test Loss: 0.371 | Test Acc: 84.98%

In [ ]:
# for CNN1d 
# Test Loss: 0.333 | Test Acc: 85.73%